_Sound_ program analyses present conservative overapproximations of program behavior.  Consider a _may-alias analysis_, which determines if two reference variables may refer to the same location in memory.  Precise may-alias analysis is undecidable, but certain kinds of imprecision are acceptable.  Often we're interested in sound analyses to support verification or semantics-preserving program transformations, so false positives are acceptable but false negatives are not.  Put another way, the worst that can come of spuriously identifying a pair of variables as potentially-aliasing is that we'd miss an opportunity to optimize our program; the worst that can come of not identifying a pair of potentially-aliasing variables as such is a program tranformation that introduces a behavior change.

By contrast, _unsound_ analyses are imprecise but not conservative:  both false positives and false negatives are possible.  These analyses are still useful for program understanding (e.g., in linters or static bug detectors) even if they are not sufficient to support program transformations.  In this post (which is also available as an interactive notebook), we'll develop an unsound analysis for Python.  The goal of the analysis we'll develop is to identify which modules a function depends on; this post will show one way to build up such a technique.  


# Background

Consider the following problem:  you'd like to enable users to automatically extract a function from a Jupyter notebook and publish it as a service.  Actually serializing a closure from a function in a given environment is not difficult, given the [`cloudpickle`](https://github.com/cloudpipe/cloudpickle) module.  But merely constructing a serialized closure isn't enough, since in general this function may require other modules to be available to run in another context.  

Therefore, we need some way to identify the modules required by a function (and, ideally, the packages that provide these modules).  Since engineering time is limited, it's probably better to have an optimistic-and-imperfect estimate and allow users to override it (by supplying additional dependencies when they publish a function) than it is to have a sound and conservative module list.

The [`modulefinder`](https://docs.python.org/3.6/library/modulefinder.html) module in the Python standard library might initially seem like an attractive option, but it is unsuitable because it operates at the level of scripts.  In order to use `modulefinder` on a single function from a notebook, we'd either have an imprecise module list (due to running the whole notebook) or we'd need to essentially duplicate a lot of its effort in order to [slice backwards](https://en.wikipedia.org/wiki/Program_slicing) from the function invocation so we could extract a suitably pruned script.

Fortunately, you can interrogate nearly any aspect of any object in a Python program, and functions are no exception.  If we could inspect the captured variables in a closure, we could identify the ones that are functions and figure out which modules they were declared in.  That would look something like this:

In [1]:
import inspect

def module_frontier(f):
  worklist = [f]
  seen = set()
  mods = set()
  for fn in worklist:
    cvs = inspect.getclosurevars(fn)
    gvars = cvs.globals
    for k, v in gvars.items():
      if inspect.ismodule(v):
        mods.add(v.__name__)
      elif inspect.isfunction(v) and id(v) not in seen:
        seen.add(id(v))
        mods.add(v.__module__)
        worklist.append(v)
      elif hasattr(v, "__module__"):
        mods.add(v.__module__)
  return list(mods)

The [`inspect`](https://docs.python.org/3/library/inspect.html#inspect.Signature.bind) module provides a friendly interface to inspecting object metadata.  In the above function, we're constructing a worklist of all of the captured variables in a given function's closure.  We're then constructing a set of all of the modules directly or transitively referred to by those captured variables, whether these are modules referred to directly, modules declaring functions referred to by captured variables, or modules declaring other values referred to by captured variables (e.g., native functions).  Note that we add any functions we find to the worklist (although we don't handle `eval` or other techniques), so we'll capture at least some of the transitive call graph in this case.

This approach seems to work pretty sensibly on simple examples:

In [2]:
import numpy as np
from numpy import dot

def f(a, b):
    return np.dot(a, b)

def g(a, b):
    return dot(a, b)

def h(a, b):
    return f(a, b)

In [3]:
{k.__name__ : module_frontier(k) for k in [f,g,h]}

{'f': ['numpy', 'numpy.core.multiarray'],
 'g': ['numpy.core.multiarray'],
 'h': ['__main__', 'numpy', 'numpy.core.multiarray']}

It also works on itself, which is a relief:

In [4]:
module_frontier(module_frontier)

['inspect']

# Problem cases

While these initial experiments are promising, we shouldn't expect that a simple approach will cover everything we might want to do.  Let's look at a (slightly) more involved example to see if it breaks down.

We'll use the k-means clustering implementation from scikit-learn to optimize some cluster centers in a model object.  We'll then capture that model object in a closure and analyze it to see what we might need to import to run it elsewhere.

In [5]:
from sklearn.cluster import KMeans
import numpy as np

data = np.random.rand(1000, 2)

model = KMeans(random_state=0).fit(data)

def km_predict_one(sample):
    sample = np.array(sample).reshape(1,-1)
    return model.predict(sample)[0]

In [6]:
km_predict_one([0.5, 0.5])

0

In [7]:
module_frontier(km_predict_one)

['numpy', 'sklearn.cluster.k_means_']

## List comprehensions

So far, so good.  Let's say we want to publish this simple model as a lighter-weight service (without a scikit-learn dependency).  We can get that by reimplementing the `predict` method from the k-means model:

In [8]:
centers = model.cluster_centers_
from numpy.linalg import norm

def km_predict_two(sample):
    _, idx = min([(norm(sample - center), idx) for idx, center in enumerate(centers)])
    return idx


In [9]:
km_predict_two([0.5, 0.5])

0

What do we get if we analyze the second method?

In [10]:
module_frontier(km_predict_two)

[]

_This is a problem!_  We'd expect that `norm` would be a captured variable in the body of `km_predict_two` (and thus that `numpy.linalg` would be listed in its module frontier), but that isn't the case.  We can inspect the closure variables:

In [11]:
inspect.getclosurevars(km_predict_two)

ClosureVars(nonlocals={}, globals={'centers': array([[ 0.48164613,  0.76963627],
       [ 0.19929899,  0.5320582 ],
       [ 0.7851443 ,  0.49048373],
       [ 0.1589274 ,  0.87394225],
       [ 0.14252367,  0.18109538],
       [ 0.83165127,  0.83706918],
       [ 0.82584485,  0.16641708],
       [ 0.47398427,  0.175393  ]])}, builtins={'min': <built-in function min>, 'enumerate': <class 'enumerate'>}, unbound=set())

We can see the cluster centers as well as the `min` function and the `enumerate` type.  But `norm` isn't in the list.  Let's dive deeper.  We can use the [`dis`](https://docs.python.org/3/library/dis.html) module (and some functionality that was introduced in Python 3.4) to inspect the Python bytecode for a given function:

In [12]:
from dis import Bytecode
for inst in Bytecode(km_predict_two):
    print("%d: %s(%s)" % (inst.offset, inst.opname, inst.argrepr))

0: LOAD_GLOBAL(min)
2: LOAD_CLOSURE(sample)
4: BUILD_TUPLE()
6: LOAD_CONST(<code object <listcomp> at 0x10dcecd20, file "<ipython-input-8-5a350184a257>", line 5>)
8: LOAD_CONST('km_predict_two.<locals>.<listcomp>')
10: MAKE_FUNCTION()
12: LOAD_GLOBAL(enumerate)
14: LOAD_GLOBAL(centers)
16: CALL_FUNCTION()
18: GET_ITER()
20: CALL_FUNCTION()
22: CALL_FUNCTION()
24: UNPACK_SEQUENCE()
26: STORE_FAST(_)
28: STORE_FAST(idx)
30: LOAD_FAST(idx)
32: RETURN_VALUE()


Ah ha!  The body of our list comprehension, which contains the call to `norm`, is a separate code object that has been stored in a constant.  Let's look at the constants for our function:

In [13]:
km_predict_two.__code__.co_consts

(None,
 <code object <listcomp> at 0x10dcecd20, file "<ipython-input-8-5a350184a257>", line 5>,
 'km_predict_two.<locals>.<listcomp>')

We can see the code object in the constant list and use `dis` to disassemble it as well:

In [14]:
for inst in Bytecode(km_predict_two.__code__.co_consts[1]):
    print("%d: %s(%s)" % (inst.offset, inst.opname, inst.argrepr))

0: BUILD_LIST()
2: LOAD_FAST(.0)
4: FOR_ITER(to 30)
6: UNPACK_SEQUENCE()
8: STORE_FAST(idx)
10: STORE_FAST(center)
12: LOAD_GLOBAL(norm)
14: LOAD_DEREF(sample)
16: LOAD_FAST(center)
18: BINARY_SUBTRACT()
20: CALL_FUNCTION()
22: LOAD_FAST(idx)
24: BUILD_TUPLE()
26: LIST_APPEND()
28: JUMP_ABSOLUTE()
30: RETURN_VALUE()


Once we've done so, we can see that the list comprehension has loaded `norm` from a global, which we can then resolve and inspect:

In [15]:
km_predict_two.__globals__["norm"]

<function numpy.linalg.linalg.norm>

In [16]:
_.__module__

'numpy.linalg.linalg'

## Nested functions and lambda expressions

We can see a similar problem if we look at a function with local definitions (note that there is no need for the nesting in this example other than to expose a limitation of our technique):

In [17]:
import sys

def km_predict_three(sample):
    # unnecessary nested function
    def find_best(sample):
        (n, i) = (sys.float_info.max, -1)
        for idx, center in enumerate(centers):
            (n, i) = min((n, i), (norm(sample - center), idx))
        return i
    return find_best(sample)

km_predict_three([0.5, 0.5])

0

In [18]:
module_frontier(km_predict_three)

[]

In this case, we can see that Python compiles these nested functions in essentially the same way it compiles the bodies of list comprehensions:

In [19]:
from dis import Bytecode
for inst in Bytecode(km_predict_three):
    print("%d: %s(%s)" % (inst.offset, inst.opname, inst.argrepr))

0: LOAD_CONST(<code object find_best at 0x10dcf7390, file "<ipython-input-17-e19a1ac37885>", line 5>)
2: LOAD_CONST('km_predict_three.<locals>.find_best')
4: MAKE_FUNCTION()
6: STORE_FAST(find_best)
8: LOAD_FAST(find_best)
10: LOAD_FAST(sample)
12: CALL_FUNCTION()
14: RETURN_VALUE()


But we can inspect the nested function just as we did the list comprehension.  Let's do that but just look at the load instructions; we'll see that we've loaded `sys` and `norm` as we'd expect.

In [20]:
from dis import Bytecode
for inst in [op for op in Bytecode(km_predict_three.__code__.co_consts[1]) if "LOAD_" in op.opname]:
    print("%d: %s(%s)" % (inst.offset, inst.opname, inst.argrepr))

0: LOAD_GLOBAL(sys)
2: LOAD_ATTR(float_info)
4: LOAD_ATTR(max)
6: LOAD_CONST(-1)
16: LOAD_GLOBAL(enumerate)
18: LOAD_GLOBAL(centers)
32: LOAD_GLOBAL(min)
34: LOAD_FAST(n)
36: LOAD_FAST(i)
40: LOAD_GLOBAL(norm)
42: LOAD_FAST(sample)
44: LOAD_FAST(center)
50: LOAD_FAST(idx)
66: LOAD_FAST(i)


Predictably, we can also see a similar problem if we analyze a function with lambda expressions:

In [21]:
def km_predict_four(sample):
    _, idx = min(map(lambda tup: (norm(sample - tup[1]), tup[0]), enumerate(centers)))
    return idx

km_predict_four([0.5, 0.5])

0

In [22]:
module_frontier(km_predict_four)

[]

## Explicit imports

Let's look at what happens when we import modules inside the function we're analyzing.  Because of the semantics of `import` in Python, the module dependency list for this function will depend on whether or not we've already imported `numpy` and `sys` in the global namespace.  If we have, we'll get a reasonable module list; if we haven't, we'll get an empty module list.  (If you're running this code in a notebook, you can try it out by restarting the kernel, re-executing the cell with the definition of `module_frontier`, and then executing this cell.)

In [100]:
def km_predict_five(sample):
    import numpy
    import sys
    from numpy.linalg import norm
    from sys.float_info import max as MAX_FLOAT
    
    (n, i) = (MAX_FLOAT, -1)
    for idx, center in enumerate(centers):
        (n, i) = min((n, i), (norm(sample - center), idx))
    return i

module_frontier(km_predict_five)

['builtins',
 'sys',
 'numpy.core._methods',
 'numpy.core.umath',
 'numpy.core.multiarray',
 'numpy.linalg.linalg',
 'numpy.core.numerictypes',
 'numpy.lib.type_check',
 'numpy.linalg._umath_linalg',
 'numpy',
 'numpy.core.numeric',
 'numpy.core.fromnumeric']

In [111]:
from dis import Bytecode
for inst in Bytecode(km_predict_five):
    print("%d: %s(%r)" % (inst.offset, inst.opname, inst.argval))

0: LOAD_CONST(0)
2: LOAD_CONST(None)
4: IMPORT_NAME('numpy')
6: STORE_FAST('numpy')
8: LOAD_CONST(0)
10: LOAD_CONST(None)
12: IMPORT_NAME('sys')
14: STORE_FAST('sys')
16: LOAD_CONST(0)
18: LOAD_CONST(('norm',))
20: IMPORT_NAME('numpy.linalg')
22: IMPORT_FROM('norm')
24: STORE_FAST('norm')
26: POP_TOP(None)
28: LOAD_CONST(0)
30: LOAD_CONST(('max',))
32: IMPORT_NAME('sys.float_info')
34: IMPORT_FROM('max')
36: STORE_FAST('MAX_FLOAT')
38: POP_TOP(None)
40: LOAD_FAST('MAX_FLOAT')
42: LOAD_CONST(-1)
44: ROT_TWO(None)
46: STORE_FAST('n')
48: STORE_FAST('i')
50: SETUP_LOOP(102)
52: LOAD_GLOBAL('enumerate')
54: LOAD_GLOBAL('centers')
56: CALL_FUNCTION(1)
58: GET_ITER(None)
60: FOR_ITER(100)
62: UNPACK_SEQUENCE(2)
64: STORE_FAST('idx')
66: STORE_FAST('center')
68: LOAD_GLOBAL('min')
70: LOAD_FAST('n')
72: LOAD_FAST('i')
74: BUILD_TUPLE(2)
76: LOAD_FAST('norm')
78: LOAD_FAST('sample')
80: LOAD_FAST('center')
82: BINARY_SUBTRACT(None)
84: CALL_FUNCTION(1)
86: LOAD_FAST('idx')
88: BUILD_TUPLE(2)
9

We can see this more clearly by importing a module in a function's scope that we haven't imported into the global namespace:

In [24]:
def example_six():
    import json
    return json.loads("{'this-sure-is': 'confusing'}")

module_frontier(example_six)

[]

In [25]:
inspect.getclosurevars(example_six)

ClosureVars(nonlocals={}, globals={}, builtins={}, unbound={'loads', 'json'})

`json` is an unbound variable (since it isn't bound in the enclosing environment of the closure).  If it were bound in the global namespace, however, the `json` we're referring to in `example_six` would be captured as a global variable:

In [26]:
import json

module_frontier(example_six)

['json']

In [27]:
inspect.getclosurevars(example_six)

ClosureVars(nonlocals={}, globals={'json': <module 'json' from '/Users/willb/anaconda/lib/python3.6/json/__init__.py'>}, builtins={}, unbound={'loads'})

Obviously, we'd like to return the same module-dependency results for functions that import modules locally independently of whether those modules have been imported into the global namespace.  We can look at the bytecode for this function to see what instructions might be relevant:

In [28]:
from dis import Bytecode
for inst in Bytecode(example_six):
    print("%d: %s(%r)" % (inst.offset, inst.opname, inst.argval))

0: LOAD_CONST(0)
2: LOAD_CONST(None)
4: IMPORT_NAME('json')
6: STORE_FAST('json')
8: LOAD_FAST('json')
10: LOAD_ATTR('loads')
12: LOAD_CONST("{'this-sure-is': 'confusing'}")
14: CALL_FUNCTION(1)
16: RETURN_VALUE(None)


# Solving problems

To address the cases that the closure-inspecting approach misses, we can inspect the bytecode of each function. (We could also inspect abstract syntax trees, using the [`ast`](https://docs.python.org/3.6/library/ast.html) module, but in general it's easier to do this sort of work with a lower-level, more regular representation.  ASTs have more special cases.)

Python bytecode is stack-based, meaning that each instruction may take one or more arguments from the stack (in addition to explicit arguments encoded in the instruction).  For a more involved analysis, we'd probably want to convert Python bytecode to a representation with explicit operands (like three-address code; see [section 2 of Vallée-Rai et al.](https://pdfs.semanticscholar.org/22df/3c0d055b7f65871068dfcd83d10f0a4fe2e4.pdf) for a reasonable approach), but let's see how far we can get by just operating on bytecode.

## Identifying interesting bytecodes

We know from the problem cases we examined earlier that we need to worry about a few different kinds of bytecode instructions to find some of the modules that our `inspect`-based approach missed:

- `LOAD_CONST` instructions that load code objects (e.g., list comprehension bodies, lambda expressions, or nested functions);
- other `LOAD_` instructions that might do the same; and
- `IMPORT_NAME` instructions that import a module into a function's namespace.

Let's extend our technique to also inspect relevant bytecodes.  We'll see how far we can get by just looking at bytecodes in isolation (without modeling the stack or value flow).

In [128]:
def interesting(inst):
    from types import CodeType, FunctionType, ModuleType
    from importlib import import_module
    from functools import reduce
    
    if inst.opname == "IMPORT_NAME":
        # XXX
        path = inst.argval.split(".")
        path[0] = import_module(path[0])
        
        return ("module", reduce(lambda x, a: getattr(x, a), path))
    if inst.opname == "LOAD_GLOBAL":
        if inst.argval in globals() and type(globals()[inst.argval]) in [CodeType, FunctionType]:
            return ("code", globals()[inst.argval])
        if inst.argval in globals() and type(globals()[inst.argval]) == ModuleType:
            return ("module", globals()[inst.argval])
        else:
            return None
    if "LOAD_" in inst.opname and type(inst.argval) in [CodeType, FunctionType]:
        return ("code", inst.argval)
    return None

def mf_revised(f):
  worklist = [f]
  seen = set()
  mods = set()
  for fn in worklist:
    codeworklist = [fn]
    cvs = inspect.getclosurevars(fn)
    gvars = cvs.globals
    for k, v in gvars.items():
        if inspect.ismodule(v):
            mods.add(v.__name__)
        elif inspect.isfunction(v) and id(v) not in seen:
            seen.add(id(v))
            mods.add(v.__module__)
            worklist.append(v)
        elif hasattr(v, "__module__"):
            mods.add(v.__module__)
    for block in codeworklist:
        for (k, v) in [interesting(inst) for inst in Bytecode(block) if interesting(inst)]:
            if k == "module":
                mods.add(v.__name__)
            elif k == "code" and id(v) not in seen:
                seen.add(id(v))
                if hasattr(v, "__module__"):
                    mods.add(v.__module__)
            if(inspect.isfunction(v)):
                worklist.append(v)
            elif(inspect.iscode(v)):
                codeworklist.append(v)
    
  return list(mods)

In [129]:
mf_revised(km_predict_one)

['numpy', 'sklearn.cluster.k_means_']

In [130]:
mf_revised(km_predict_two)

['builtins',
 'numpy.core._methods',
 'numpy.core.umath',
 'numpy.core.multiarray',
 'numpy.linalg.linalg',
 'numpy.core.numerictypes',
 'numpy.lib.type_check',
 'numpy.linalg._umath_linalg',
 'numpy',
 'numpy.core.numeric',
 'numpy.core.fromnumeric']

In [131]:
mf_revised(km_predict_three)

['builtins',
 'sys',
 'numpy.core._methods',
 'numpy.core.umath',
 'numpy.core.multiarray',
 'numpy.linalg.linalg',
 'numpy.core.numerictypes',
 'numpy.lib.type_check',
 'numpy.linalg._umath_linalg',
 'numpy',
 'numpy.core.numeric',
 'numpy.core.fromnumeric']

In [132]:
mf_revised(km_predict_four)

['builtins',
 'numpy.core._methods',
 'numpy.core.umath',
 'numpy.core.multiarray',
 'numpy.linalg.linalg',
 'numpy.core.numerictypes',
 'numpy.lib.type_check',
 'numpy.linalg._umath_linalg',
 'numpy',
 'numpy.core.numeric',
 'numpy.core.fromnumeric']

In [133]:
mf_revised(km_predict_five)

AttributeError: 'sys.float_info' object has no attribute '__name__'

In [70]:
type(getattr(__builtins__, "enumerate")).__class__

type

In [42]:
globals()

{'Bytecode': dis.Bytecode,
 'In': ['',
  'import inspect\n\ndef module_frontier(f):\n  worklist = [f]\n  seen = set()\n  mods = set()\n  for fn in worklist:\n    cvs = inspect.getclosurevars(fn)\n    gvars = cvs.globals\n    for k, v in gvars.items():\n      if inspect.ismodule(v):\n        mods.add(v.__name__)\n      elif inspect.isfunction(v) and id(v) not in seen:\n        seen.add(id(v))\n        mods.add(v.__module__)\n        worklist.append(v)\n      elif hasattr(v, "__module__"):\n        mods.add(v.__module__)\n  return list(mods)',
  'import numpy as np\nfrom numpy import dot\n\ndef f(a, b):\n    return np.dot(a, b)\n\ndef g(a, b):\n    return dot(a, b)\n\ndef h(a, b):\n    return f(a, b)',
  '{k.__name__ : module_frontier(k) for k in [f,g,h]}',
  'module_frontier(module_frontier)',
  'from sklearn.cluster import KMeans\nimport numpy as np\n\ndata = np.random.rand(1000, 2)\n\nmodel = KMeans(random_state=0).fit(data)\n\ndef km_predict_one(sample):\n    sample = np.array(sample

In [37]:
from dis import Bytecode
for inst in Bytecode(km_predict_one):
    print("%d: %s(%s)" % (inst.offset, inst.opname, inst.argrepr))

0: LOAD_GLOBAL(np)
2: LOAD_ATTR(array)
4: LOAD_FAST(sample)
6: CALL_FUNCTION()
8: LOAD_ATTR(reshape)
10: LOAD_CONST(1)
12: LOAD_CONST(-1)
14: CALL_FUNCTION()
16: STORE_FAST(sample)
18: LOAD_GLOBAL(model)
20: LOAD_ATTR(predict)
22: LOAD_FAST(sample)
24: CALL_FUNCTION()
26: LOAD_CONST(0)
28: BINARY_SUBSCR()
30: RETURN_VALUE()


In [30]:
getattr(__builtins__, "enumerate").__module__

'builtins'

In [31]:
type(km_predict_two.__code__.co_consts[1])

code

In [32]:
import types

In [33]:
types.CodeType == type(km_predict_two.__code__.co_consts[1])

True

In [34]:
import inspect

def module_frontier(f):
  worklist = [f]
  seen = set()
  mods = set()
  for fn in worklist:
    cvs = inspect.getclosurevars(fn)
    gvars = cvs.globals
    for k, v in gvars.items():
      if inspect.ismodule(v):
        mods.add(v.__name__)
      elif inspect.isfunction(v) and id(v) not in seen:
        seen.add(id(v))
        mods.add(v.__module__)
        worklist.append(v)
      elif hasattr(v, "__module__"):
        mods.add(v.__module__)
  return list(mods)

This is ongoing work and future notebooks will cover refinements to this technique.  It has been fun to learn about the power (and limitations) of the `inspect` module -- as well as a little more about how Python compiles code blocks.